<a href="https://colab.research.google.com/github/A-R-TAQI/ml_lab/blob/main/ML_assignment_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Abdur rehman taqi**

**4SF20CI001**

## **9.2`ASSOCIATION RULES`**
**Applying Association Rules** We will create association rules using the transactions data available in the groceries.csv dataset. Each line
in the dataset is an order and contains a variable number of items. Each item in each order is separated
by a comma in the dataset.

**Loading the Dataset**

In [70]:
all_txns = []
#open the file
with open('/content/sample_data/groceries.csv') as f:
#read each line
  content = f.readlines()
#Remove white space from the beginning and end of the line
  txns = [x.strip() for x in content]
# Iterate through each line and create a list of transactions
  for each_txn in txns:
#Each transaction will contain a list of item in the transaction
    all_txns.append( each_txn.split(',') )

In [71]:
all_txns[0:5]

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product']]

***Encoding the Transactions***

In [73]:
# Import all required libraries
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
# Initialize OnehotTransactions
one_hot_encoding = TransactionEncoder()
# Transform the data into one-hot-encoding format
one_hot_txns = one_hot_encoding.fit(all_txns).transform(all_txns)
# Convert the matrix into the dataframe.
one_hot_txns_df = pd.DataFrame(one_hot_txns,columns=one_hot_encoding.columns_)

In [74]:
one_hot_txns_df.iloc[5:10, 10:20]

,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles
5,False,False,False,False,False,False,True,False,False,False
6,False,False,False,False,False,False,False,False,False,False
7,False,False,True,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False


In [75]:
one_hot_txns_df.shape

(9835, 169)

***Generating Association Rules***

In [76]:
len(one_hot_txns_df.columns)


169

In [77]:
frequent_itemsets = apriori(one_hot_txns_df,min_support=0.02,use_colnames=True)

In [78]:
frequent_itemsets.sample(10, random_state = 90)

,support,itemsets
60,0.020437,"(bottled beer, whole milk)"
52,0.033859,(sugar)
89,0.035892,"(other vegetables, tropical fruit)"
105,0.021047,"(root vegetables, tropical fruit)"
88,0.032740,"(other vegetables, soda)"
16,0.058058,(coffee)
111,0.024504,"(shopping bags, whole milk)"
36,0.079817,(newspapers)
119,0.056024,"(yogurt, whole milk)"
55,0.071683,(whipped/sour cream)


In [79]:
rules = association_rules(frequent_itemsets, # itemsets
metric="lift", # lift
min_threshold=1)

In [80]:
rules.sample(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
83,(tropical fruit),(rolls/buns),0.104931,0.183935,0.024606,0.234496,1.274886,0.005305,1.066050,0.240894
94,(sausage),(soda),0.093950,0.174377,0.024301,0.258658,1.483324,0.007918,1.113687,0.359626
119,(root vegetables),"(other vegetables, whole milk)",0.108998,0.074835,0.023183,0.212687,2.842082,0.015026,1.175091,0.727435
39,(margarine),(whole milk),0.058566,0.255516,0.024199,0.413194,1.617098,0.009235,1.268706,0.405348
27,(domestic eggs),(other vegetables),0.063447,0.193493,0.022267,0.350962,1.813824,0.009991,1.242619,0.479074


***Top Ten Rules***

In [81]:
rules.sort_values('confidence',ascending = False)[0:10]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
120,"(other vegetables, yogurt)",(whole milk),0.043416,0.255516,0.022267,0.512881,2.007235,0.011174,1.528340,0.524577
16,(butter),(whole milk),0.055414,0.255516,0.027555,0.497248,1.946053,0.013395,1.480817,0.514659
25,(curd),(whole milk),0.053279,0.255516,0.026131,0.490458,1.919481,0.012517,1.461085,0.505984
116,"(other vegetables, root vegetables)",(whole milk),0.047382,0.255516,0.023183,0.489270,1.914833,0.011076,1.457687,0.501524
115,"(root vegetables, whole milk)",(other vegetables),0.048907,0.193493,0.023183,0.474012,2.449770,0.013719,1.533320,0.622230
28,(domestic eggs),(whole milk),0.063447,0.255516,0.029995,0.472756,1.850203,0.013783,1.412030,0.490649
108,(whipped/sour cream),(whole milk),0.071683,0.255516,0.032232,0.449645,1.759754,0.013916,1.352735,0.465077
91,(root vegetables),(whole milk),0.108998,0.255516,0.048907,0.448694,1.756031,0.021056,1.350401,0.483202
51,(root vegetables),(other vegetables),0.108998,0.193493,0.047382,0.434701,2.246605,0.026291,1.426693,0.622764
32,(frozen vegetables),(whole milk),0.048094,0.255516,0.020437,0.424947,1.663094,0.008149,1.294636,0.418855


## **9.3.2 `User-Based Similarity`**
**User-Based Similarity**
We will use MovieLens dataset (see https://grouplens.org/datasets/movielens/) for finding similar users
based on common movies the users have watched and how they have rated those movies. The file ratings.
csv in the dataset contains ratings given by users. Each line in this file represents a rating given by a user
to a movie. The ratings are on the scale of 1 to 5. The dataset has the following features:
1. userId
2. movieId
3. rating
4. timestamp

In [82]:
rating_df = pd.read_csv("/content/ratings.csv")

In [83]:
rating_df.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [84]:
rating_df.drop('timestamp', axis = 1, inplace = True )

In [85]:
len( rating_df.userId.unique() )

610

In [86]:
len( rating_df.movieId.unique() )

9724

In [87]:
user_movies_df = rating_df.pivot( index='userId',
columns='movieId',
values = "rating"
).reset_index(drop=True)
user_movies_df.index=rating_df.userId.unique()

In [88]:
user_movies_df.iloc[0:5, 0:15]

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
user_movies_df.fillna( 0, inplace = True)
user_movies_df.iloc[0:5, 0:10]

movieId,1,2,3,4,5,6,7,8,9,10
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


***Calculating Cosine Similarity between Users***

In [90]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
user_sim = 1 - pairwise_distances( user_movies_df.values,metric="cosine")
#Store the results in a dataframe
user_sim_df = pd.DataFrame( user_sim )
#Set the index and column names to user ids (0 to 671)
user_sim_df.index = rating_df.userId.unique()
user_sim_df.columns = rating_df.userId.unique()

In [91]:
user_sim_df.iloc[0:5, 0:5]

,1,2,3,4,5
1,1.000000,0.027283,0.059720,0.194395,0.129080
2,0.027283,1.000000,0.000000,0.003726,0.016614
3,0.059720,0.000000,1.000000,0.002251,0.005020
4,0.194395,0.003726,0.002251,1.000000,0.128659
5,0.129080,0.016614,0.005020,0.128659,1.000000


In [92]:
user_sim_df.shape

(610, 610)

In [93]:
np.fill_diagonal( user_sim, 0 )
user_sim_df.iloc[0:5, 0:5]

,1,2,3,4,5
1,0.000000,0.027283,0.059720,0.194395,0.129080
2,0.027283,0.000000,0.000000,0.003726,0.016614
3,0.059720,0.000000,0.000000,0.002251,0.005020
4,0.194395,0.003726,0.002251,0.000000,0.128659
5,0.129080,0.016614,0.005020,0.128659,0.000000


**Filtering Similar Users**

In [94]:
user_sim_df.idxmax(axis=1)[0:5]

1    266
2    366
3    313
4    391
5    470
dtype: int64

In [95]:
user_sim_df.iloc[1:2, 330:340]

,331,332,333,334,335,336,337,338,339,340
2,0.199366,0.073652,0.050674,0.053668,0.073991,0.046544,0.018408,0.074145,0.111447,0.03063


**loading the movies**

In [96]:
movies_df = pd.read_csv("/content/sample_data/movies.csv")

In [97]:
movies_df[0:5]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [98]:
movies_df.drop( 'genres', axis = 1, inplace = True )

**Finding Common Movies of Similar Users**

In [99]:
def get_user_similar_movies( user1, user2 ):
# Inner join between movies watched between two users will give
# the common movies watched.
  common_movies = rating_df[rating_df.userId == user1].merge(
  rating_df[rating_df.userId == user2],
  on = "movieId",
  how = "inner" )
  # join the above result set with movies details
  return common_movies.merge( movies_df, on = 'movieId' )

In [100]:
common_movies = get_user_similar_movies( 2, 331 )

In [101]:
common_movies[(common_movies.rating_x >= 4.0) & ((common_movies.rating_y >= 4.0))]

,userId_x,movieId,rating_x,userId_y,rating_y,title
1,2,3578,4.0,331,4.5,Gladiator (2000)
2,2,6874,4.0,331,4.0,Kill Bill: Vol. 1 (2003)
3,2,58559,4.5,331,5.0,"Dark Knight, The (2008)"
4,2,79132,4.0,331,4.0,Inception (2010)
9,2,112552,4.0,331,5.0,Whiplash (2014)
11,2,122882,5.0,331,5.0,Mad Max: Fury Road (2015)


In [102]:
common_movies = get_user_similar_movies( 2, 337 )
common_movies

,userId_x,movieId,rating_x,userId_y,rating_y,title
0,2,318,3.0,337,5.0,"Shawshank Redemption, The (1994)"


## **` 9.3.3 Item-Based Similarity`**
If two movies, movie A and movie B, have been watched by several users and rated very similarly, then
movie A and movie B can be similar in taste. In other words, if a user watches movie A, then he or she is
very likely to watch B and vice versa.

 **Calculating Cosine Similarity between Movie**

In [103]:
rating_mat = rating_df.pivot(index='movieId', columns='userId',values = "rating").reset_index(drop = True)
# Fill all NaNs with 0
rating_mat.fillna(0, inplace = True)
# Find the correlation between movies
movie_sim = 1 - pairwise_distances(rating_mat.values,metric="correlation")
# Fill the diagonal with 0, as it repreresents the auto-correlation
# of movies
movie_sim_df = pd.DataFrame( movie_sim )

In [104]:
movie_sim_df.iloc[0:5, 0:5]

,0,1,2,3,4
0,1.000000,0.231327,0.173213,-0.028917,0.192474
1,0.231327,1.000000,0.191945,0.071269,0.200526
2,0.173213,0.191945,1.000000,0.067143,0.370171
3,-0.028917,0.071269,0.067143,1.000000,0.167910
4,0.192474,0.200526,0.370171,0.167910,1.000000


In [105]:
movie_sim_df.shape

(9724, 9724)

**Finding Most Similar Movies**

In [106]:
def get_similar_movies( movieid, topN = 5 ):
# Get the index of the movie record in movies_df
  movieidx = movies_df[movies_df.movieId == movieid].index[0]
  movies_df['similarity'] = movie_sim_df.iloc[movieidx]
  top_n = movies_df.sort_values( ["similarity"], ascending =
  False )[0:topN]
  return top_n

In [107]:
movies_df[movies_df.movieId == 858]

,movieId,title
659,858,"Godfather, The (1972)"


In [108]:
get_similar_movies(858)

,movieId,title,similarity
659,858,"Godfather, The (1972)",1.000000
921,1220,"Blues Brothers, The (1980)",0.769390
913,1212,"Third Man, The (1949)",0.560246
895,1192,Paris Is Burning (1990),0.496048
827,1088,Dirty Dancing (1987),0.442128


In [109]:
movies_df[movies_df.movieId == 231]

,movieId,title,similarity
197,231,Dumb & Dumber (Dumb and Dumber) (1994),0.095286


In [110]:
get_similar_movies(231)

,movieId,title,similarity
197,231,Dumb & Dumber (Dumb and Dumber) (1994),1.000000
302,344,Ace Ventura: Pet Detective (1994),0.582137
138,165,Die Hard: With a Vengeance (1995),0.465081
291,333,Tommy Boy (1995),0.464892
126,153,Batman Forever (1995),0.432095
